In [51]:
import polars as pl
from pathlib import Path
import glob


# ----------------------------------------------------------
# Notebook settings 
# ----------------------------------------------------------
pl.Config.set_tbl_cols(30)
pl.Config.set_tbl_rows(50)



# ----------------------------------------------------------
# load and transform the tables
# ----------------------------------------------------------

files = glob.glob('*.csv')              # it gives all the csv files in current directory
swift_codes = Path.cwd() / files[0]     # absoulte file path
transactions = Path.cwd() / files[1]    # --


# load the data instead of read_csv
df_transactions = ( pl.read_csv(transactions, has_header=True, sep=",")
                    .with_column(pl.col("Sort Code").str.replace_all(r"-", ""))
                  )


df_swift = ( pl.read_csv(swift_codes, has_header=True, sep= ',') )

# rename all cols to lowercase
df_transactions.columns = list(map(lambda x: x.lower().replace(' ', '_'), df_transactions.columns))
df_swift.columns = list(map(lambda x: x.lower().replace(' ', '_'), df_swift.columns))


# joining the tables on Bank, and creating the country sacalar value
df = ( df_transactions.join(df_swift, left_on="bank", right_on="bank", how="inner") 
                      .with_column(pl.lit("GB").alias("country")) )


# adding the IBAN column
df = df.with_column( (pl.col("country") + pl.col("check_digits") + 
                      pl.col("swift_code") + pl.col("sort_code").cast(pl.Utf8) +
                      pl.col("account_number").cast(pl.Utf8))
                      .alias("IBAN"))  
                        

# Selecting only necessary fields
df = df.select(["transaction_id", "IBAN"])



# ----------------------------------------------------------
# output to csv file
# ----------------------------------------------------------

output_path = Path.cwd() / 'output-py.csv'
df.write_csv(output_path, has_header=True, sep=",")


